# TRAIN MODEL CNN 

In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks


# Đường dẫn đến thư mục chứa dữ liệu huấn luyện
data_dir = 'image'

# Kích thước của ảnh (48x48 pixels)
img_size = (48, 48)

# Tạo đối tượng ImageDataGenerator với tỉ lệ chia là 8:2
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

# Tạo dữ liệu huấn luyện từ thư mục
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=32,
    class_mode='categorical',
    subset='training'  # Sử dụng 80% dữ liệu làm tập huấn luyện
)

# Tạo dữ liệu kiểm thử từ thư mục
validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # Sử dụng 20% dữ liệu làm tập kiểm thử
)

# Số lượng lớp
num_classes = len(train_generator.class_indices)

# Xây dựng mô hình CNN (giống như ví dụ trước)
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.5))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Dropout(0.75))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))  # Số lớp đầu ra

model_checkpoint = callbacks.ModelCheckpoint(
    filepath='best_model1.h5',
    save_best_only=True,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    verbose=1
)

# Compile the model with the new callbacks list
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model with the callbacks list
history = model.fit(train_generator, epochs=50, validation_data=validation_generator, callbacks=[model_checkpoint])

# Load the best model based on validation accuracy
best_model = models.load_model('best_model1.h5')

# Print information about the best model
best_model.summary()

# Print the keys of history to access different metrics
print(history.history.keys())



Found 4992 images belonging to 5 classes.
Found 1248 images belonging to 5 classes.
Epoch 1/50
154/156 [============================>.] - ETA: 0s - loss: 1.6104 - accuracy: 0.1942
Epoch 1: val_accuracy improved from -inf to 0.20593, saving model to best_model1.h5
156/156 [==============================] - 5s 24ms/step - loss: 1.6104 - accuracy: 0.1939 - val_loss: 1.6092 - val_accuracy: 0.2059
Epoch 2/50
154/156 [============================>.] - ETA: 0s - loss: 1.5632 - accuracy: 0.2585
Epoch 2: val_accuracy improved from 0.20593 to 0.31971, saving model to best_model1.h5
156/156 [==============================] - 3s 22ms/step - loss: 1.5613 - accuracy: 0.2592 - val_loss: 1.5390 - val_accuracy: 0.3197
Epoch 3/50
154/156 [============================>.] - ETA: 0s - loss: 1.0937 - accuracy: 0.4556
Epoch 3: val_accuracy improved from 0.31971 to 0.49038, saving model to best_model1.h5
156/156 [==============================] - 3s 21ms/step - loss: 1.0934 - accuracy: 0.4571 - val_loss: 1.31

In [15]:
# Import các thư viện cần thiết
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
image_size = 48
batch_size = 32
# Đọc lại mô hình đã được lưu
model = load_model('best_model1.h5')

# Đường dẫn đến thư mục chứa dữ liệu kiểm thử
test_data_dir = 'C:/Users/Admin/Desktop/train-20240111T182556Z-001/train'

# Sử dụng ImageDataGenerator để tạo dữ liệu kiểm thử
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='categorical',  # Nếu bạn đang làm việc với nhiều lớp
    shuffle=False  # Để giữ thứ tự của các ảnh giống với thứ tự trong thư mục
)

# Đánh giá mô hình trên tập kiểm thử
scores = model.evaluate(test_generator)

# In ra score cho từng lớp
for i in range(len(model.metrics_names)):
    print(f"Test {model.metrics_names[i]}: {scores[i]}")


Found 1240 images belonging to 5 classes.
39/39 [==============================] - 1s 16ms/step - loss: 1.5083 - accuracy: 0.6702
Test loss: 1.508269190788269
Test accuracy: 0.6701613068580627


# SAVE MODEL

In [3]:
from joblib import dump
dump(model, 'cnn_model.h5')

['cnn_model.h5']

# TEST

In [21]:
from tensorflow.keras.preprocessing import image

model1 = load_model('best_model1.h5')
# Đường dẫn đến ảnh bạn muốn dự đoán
image_path = "test.jpeg"

# Load ảnh và chuyển về kích thước và định dạng phù hợp
img = image.load_img(image_path, target_size=(48, 48))
img_array = image.img_to_array(img)
print(img_array.shape)
img_array = np.expand_dims(img_array, axis=0)  # Thêm một chiều để tạo batch

# Chuẩn hóa giá trị pixel về khoảng [0, 1]
img_array /= 255.0

print(img_array.shape)

# Sử dụng mô hình để dự đoán
predictions = model1.predict(img_array)

# Lấy nhãn có xác suất cao nhất
predicted_label = np.argmax(predictions)

# Hiển thị kết quả
print(f"Predicted label: {predicted_label}")

(48, 48, 3)
(1, 48, 48, 3)
1/1 [==============================] - 0s 133ms/step
Predicted label: 2


In [87]:
%pip install --upgrade gradio

Note: you may need to restart the kernel to use updated packages.


# GRADIO INTERFACE

In [2]:
import gradio as gr
import numpy as np
from PIL import Image
from joblib import load
import dlib
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

model = load_model('best_model1.h5')

emotion_labels = {
    0: "Angry",
    1: "Fear",
    2: "Happy",
    3: "Neutral",
    4: "Sad",
}

# Load dlib's pre-trained face detector
detector = dlib.get_frontal_face_detector()

def predict_emotion(image):
    # Convert Gradio Image to NumPy array
    img_array = np.array(image)

    # Convert to grayscale
    img = Image.fromarray(img_array).resize((48, 48))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add a dimension to create a batch

    # Normalize pixel values to the range [0, 1]
    img_array /= 255.0

    # Make prediction using CNN model
    predictions = model.predict(img_array)
    predicted_label = np.argmax(predictions)

    # Detect faces using dlib
    faces = detector(img_array[0], 1)

    if len(faces) == 0:
        cropped_face = img_array
    else:
        # Assuming there's only one face detected, you can modify this if needed
        x, y, w, h = faces[0].left(), faces[0].top(), faces[0].width(), faces[0].height()
        cropped_face = img_array[:, y:y+h, x:x+w, :]

    print("Input image shape:", img_array.shape)
    print("Processed image shape:", cropped_face.shape)

    return emotion_labels[predicted_label], cropped_face

iface = gr.Interface(
    fn=predict_emotion,
    inputs="image",
    outputs=["text", "image"],  
    title="Emotion Detection",
    description="Upload an image and predict the emotion."
)

iface.launch()




Running on local URL:  http://127.0.0.1:7882

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\gradio\queueing.py", line 495, in call_prediction
    output = await route_utils.call_process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\gradio\route_utils.py", line 232, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\gradio\blocks.py", line 1561, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\Loca